In [1]:
"""Test the radiosity.Radiosity module."""
import numpy as np
import pyfar as pf
import sparrowpy as sp
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline
%matplotlib widget

# ruff: noqa: ERA001

Define the parameters and geometry for the simulation.

In [2]:
# room dimensions
import math

# R = 1.5
planeZ = 5  # height of the plane in space
height = 1  # height of the source
min_XY = 0.01  # plane dimensions in meters
max_XY = 100

check_patch_size = 0  # meant as check_plane_dimensions = not check_patch_size
resolution_of_sim = 40  # resolution of the range of simulated geometry
res = max(10, resolution_of_sim)  # min sim res is 10
patch_max_size_structure_inf_plane = 0.3  # cutoff for patch size for large pl
patch_max_size = 2
patch_min_size = 0.05
ir_length_s = 1  # min length otherwise out of bounds refl for first order
sampling_rate = 1000
max_order_k = 1
speed_of_sound = 346.18
absorption = 1  # param default 0.1

# create geometry
# for logspace logspace(-2, 6.64, res, base=2) # length for 0.25 to 100m
min_XY = math.log2(min_XY)
max_XY = math.log2(max_XY)

# circleRes = np.arange(0, 2 * np.pi * R, 0.1)  # circle of 1.5 meters
# probeDims = np.column_stack(
#    (np.cos(circleRes), np.sin(circleRes), [5] * len(circleRes)),
# )
debug__size_plane = np.logspace(min_XY, max_XY, res, base=2)
index_of_8m = (np.abs(debug__size_plane - 8)).argmin()  # 0.25m-100m res30 ~8m [17]

elements = [  # [17/index_of_8m] corner 8.37m
    sp.geometry.Polygon(
        np.array(
            [
                [0, 0, planeZ],
                [0, i, planeZ],
                [i, i, planeZ],
                [i, 0, planeZ],
            ],
        ),
        np.array([1, 0, 0]),
        np.array([0, 0, 1]),
    )
    for i in np.logspace(min_XY, max_XY, res, base=2)  # length for 0.25 to 100m
]

source_pos = [
    np.array([i / 2, i / 2, planeZ + height]) for i in np.logspace(min_XY, max_XY, res, base=2)
]
sources = [
    sp.geometry.SoundSource(spos, np.array([0, 1, 0]), np.array([0, 0, 1]))
    for spos in source_pos
]
receiver_pos = source_pos  # sub to change

Calculate old slow approach for comparison.

In [3]:
debug__patch_sizeOrig = np.logspace(min_XY, max_XY, res, base=2)[index_of_8m] / 50
debug__patch_sizeNew1 = [min(i, 0.5) for i in (np.logspace(min_XY, max_XY, res, base=2)) / 50]
debug__patch_sizeNew1index = min(
    np.logspace(min_XY, max_XY, res, base=2)[index_of_8m] / 50,
    0.5,
)

start = datetime.now()
# init object
patch_values = np.arange(
    patch_max_size,
    patch_min_size,
    -(patch_max_size - patch_min_size) / res,
)

if check_patch_size:
    radiPlanes = [
        sp.DRadiosityFast.from_polygon(
            [elements[index_of_8m]],
            i,
        )
        for i in patch_values
    ]
else:
    radiPlanes = [
        sp.DRadiosityFast.from_polygon(
            [element],  # singular element
            (float)(
                np.min(
                    [
                        (np.logspace(min_XY, max_XY, res, base=2)[index] ** 2 / 2000) ** 0.5,
                        patch_max_size_structure_inf_plane,
                    ],
                ),
            ),
            # 0.3,
            # (float)(min(((np.logspace(min_XY, max_XY, res, base=2)[index]) / 50), 0.5))
        )
        for index, element in enumerate(elements)
    ]

# run simulation
source_ = pf.Coordinates(0, 0, 1)
receiver_ = pf.Coordinates(0, 0, 1)
frequencies = np.array([1000])
data_scattering = pf.FrequencyData(
    np.ones((source_.csize, receiver_.csize, frequencies.size)),
    frequencies,
)  # create directional scattering data (totally diffuse)


# Initialize histograms
histograms = np.zeros_like(radiPlanes)
for indexForRadiPlanes, sim in enumerate(radiPlanes):
    # set directional scattering data
    sim.set_wall_scattering(
        [0],  # np.arange(len(elements)).tolist() here only one element as in one plane per sim
        data_scattering,
        source_,
        receiver_,
    )

    # set air absorption
    sim.set_air_attenuation(
        pf.FrequencyData(
            np.zeros_like(data_scattering.frequencies),
            data_scattering.frequencies,
        ),
    )

    # set absorption coefficient
    sim.set_wall_absorption(
        [0],  # same as with set_wall_scattering only for one plane/wall per sim
        pf.FrequencyData(
            np.zeros_like(data_scattering.frequencies),
            data_scattering.frequencies,
        ),
    )

    # calculate from factors including directivity and absorption
    sim.bake_geometry()  # ff_method="universal", algorithm="order"

    if check_patch_size:
        # [i.run(sources[index_of_8m]) for i in radiPlanes]
        sim.init_source_energy(
            source_pos[index_of_8m],
        )
    else:
        # [i.run(sources[min(index, len(sources) - 1)]) for index, i in enumerate(radiPlanes)]
        sim.init_source_energy(
            source_pos[indexForRadiPlanes],
        )

    sim.calculate_energy_exchange(
        # receiver_pos[dependent on relevant index #1 or #radiPlanes], sim knows receiver_pos
        speed_of_sound=speed_of_sound,
        histogram_time_resolution=1 / sampling_rate,
        histogram_length=ir_length_s,
        max_depth=max_order_k,
    )

    # gather energy at receiver  # receiver not as geometry #i.energy_at_receiver
    if check_patch_size:
        histograms += sim.collect_receiver_energy(
            receiver_pos=receiver_pos[index_of_8m],
            speed_of_sound=speed_of_sound,
            histogram_time_resolution=1 / sampling_rate,
        )
    else:
        histograms += sim.collect_receiver_energy(
            receiver_pos=receiver_pos[indexForRadiPlanes],
            speed_of_sound=speed_of_sound,
            histogram_time_resolution=1 / sampling_rate,
        )


# stop timer
delta = datetime.now() - start
print(f"Time elapsed: {delta}")
slow_time_s = (delta.seconds * 1e6 + delta.microseconds) / 1e6

C:\Users\klingelnberg\BA\sparrowpy\sparrowpy\radiosity_fast\form_factor.py:198: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float64, 1, 'A', False, aligned=True), Array(float64, 1, 'C', False, aligned=True))
  form_factors[i,j] = calc_form_factor(


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(array(float64, 1d, C), Tuple(array(int64, 1d, C), int64))
 
There are 22 candidate implementations:
[1m  - Of which 20 did not match due to:
  Overload of function 'getitem': File: <numerous>: Line N/A.
    With argument(s): '(array(float64, 1d, C), Tuple(array(int64, 1d, C), int64))':[0m
[1m   No match.[0m
[1m  - Of which 2 did not match due to:
  Overload in function 'GetItemBuffer.generic': File: numba\core\typing\arraydecl.py: Line 211.
    With argument(s): '(array(float64, 1d, C), Tuple(array(int64, 1d, C), int64))':[0m
[1m   Rejected as the implementation raised a specific error:
     NumbaTypeError: [1mcannot index array(float64, 1d, C) with 2 indices: Tuple(array(int64, 1d, C), int64)[0m[0m
  raised from 

Diffuse and specular Intensity with plotting

In [ ]:
# Signal
I_diffuse_planes = [
    pf.Signal(i, sampling_rate=sampling_rate) for i in histograms
]

I_specular = 1 / (4 * np.pi * (height * 2) ** 2)
factors = [np.sum(i.time[:, :]) / I_specular for i in I_diffuse_planes]

# Plot the factors
plt.figure()
if check_patch_size:
    plt.plot(patch_values, factors, label="factor")
    plt.gca().invert_xaxis()
    plt.xlabel("patch size")
else:
    plt.plot(np.logspace(min_XY, max_XY, res, base=2), factors, label="factor")
    plt.xlabel("plane width/length")
    plt.xscale("log")

    plt.figure()
    plt.plot(
        np.arctan(np.logspace(min_XY, max_XY, res, base=2) / 2 / height)
        / np.pi
        * 180,
        [(i / 2) for i in factors],
        label="factor",
    )
    plt.xlabel("plane edge angle to mid positioned source=receiver")
    plt.xscale("linear")

plt.ylabel("I_diffuse / I_specular")
plt.legend()
plt.show()

Find signal points

In [ ]:
%load_ext watermark
%watermark -v -m -iv